In [1]:
import logging
import os
import sys
import time

import numpy as np
import matplotlib.pyplot as plt
import pykeen
from pykeen.kge_models import Region

import json

In [2]:
%matplotlib inline

In [3]:
logging.basicConfig(level=logging.INFO)
logging.getLogger('pykeen').setLevel(logging.INFO)

In [4]:
print(sys.version)

3.6.8 |Anaconda, Inc.| (default, Feb 21 2019, 18:30:04) [MSC v.1916 64 bit (AMD64)]


In [5]:
print(time.asctime())

Fri Apr 26 06:22:30 2019


In [6]:
print(pykeen.get_version())

0.0.26-dev


Check which hyper-parameters are required by Region:

In [7]:
Region.hyper_params

['embedding_dim',
 'margin_loss',
 'learning_rate',
 'normalization_of_entities',
 'init_radius',
 'reg_lambda',
 'loss_type',
 'single_pass',
 'neg_factor',
 'region_type']

Define output directory:

In [8]:
TEST = False

today = time.localtime()
output_directory = 'D:/uni/master/models/region_hpo/opt_{}/'.format(today.tm_year + today.tm_mon + today.tm_mday)

train_path = [
    'D:/uni/master/data/YAGO39K/Train/triple2id.tsv',
    'D:/uni/master/data/YAGO39K/Train/instanceOf2id.tsv',
    'D:/uni/master/data/YAGO39K/Train/subClassOf2id.tsv'
]
val_path =  [
    'D:/uni/master/data/YAGO39K/Valid/triple2id_positive.tsv',
    'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_positive.tsv',
    'D:/uni/master/data/YAGO39K/Valid/subClassOf2id_positive.tsv'
]
neg_val_path = [
    'D:/uni/master/data/YAGO39K/Valid/triple2id_negative.tsv',
    'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_negative.tsv',
    'D:/uni/master/data/YAGO39K/Valid/subClassOf2id_negative.tsv'
]

test_path =  [
    'D:/uni/master/data/YAGO39K/Test/triple2id_positive.tsv',
    'D:/uni/master/data/YAGO39K/Test/instanceOf2id_positive.tsv',
    'D:/uni/master/data/YAGO39K/Test/subClassOf2id_positive.tsv'
]

neg_test_path = [
    'D:/uni/master/data/YAGO39K/Test/triple2id_negative.tsv',
    'D:/uni/master/data/YAGO39K/Test/instanceOf2id_negative.tsv',
    'D:/uni/master/data/YAGO39K/Test/subClassOf2id_negative.tsv'
]


if TEST:
    output_directory = 'D:/uni/master/models/test/'
    train_path = [
        'D:/uni/master/data/debug/triple2id.tsv'
    ]
    test_path =  [
        'D:/uni/master/data/debug/triple2id_positive_test.tsv'
    ]
    neg_test_path = [
        'D:/uni/master/data/YAGO39K/Test/triple2id_negative.tsv',
    ]
    val_path =  [
        'D:/uni/master/data/debug/triple2id_positive_test.tsv'
    ]
    neg_val_path = [
        'D:/uni/master/data/YAGO39K/Valid/triple2id_negative.tsv',
    ]

config = dict(
    metrics                     = ['prediction'],
    training_set_path           = train_path,
    test_set_path               = val_path,
    neg_test_set_path           = neg_val_path,
    execution_mode              = 'Training_mode',
    random_seed                 = 2,
    kg_embedding_model_name     = 'Region',
    region_type                 = '2',
    embedding_dim               = 50,
    normalization_of_entities   = 2,
    init_radius                 = 5,
    reg_lambda                  = 0.02,
    loss_type                   = 'MRL',
    margin_loss                 = 0.0,  # für WS, max 0.5
    single_pass                 = True,
    neg_factor                  = 3,
    learning_rate               = 1,
    num_epochs                  = 10 if not TEST else 20,  
    batch_size                  = 2048,
    test_batch_size             = 4096,
    filter_negative_triples     = True,
    preferred_device            = 'gpu'
)

Define hyper-parameters inclduing the evaluation information:
 * Define the hyper-parameter optimization mode: **HPO_mode**
 * Define for each hyper-parameter a list of values (single valued lists are possible)
 * Define the number of hyper-parameter optimization iterations: **maximum_number_of_hpo_iters**
 * Define the path to the test set (**test_set_path**) or define the ratio of the training set that should be used as test set (**test_set_ratio**)
 * Define whether you want to compute the metrics (mean rank and hits@k) in raw or in a filtered setting: **filter_negative_triples**

In [9]:
config = dict(
    training_set_path           = train_path,
    test_set_path               = val_path,
    neg_test_set_path           = neg_val_path,
    execution_mode              = 'HPO_mode', # Define hyper-parameter optimization mode
    random_seed                 = 2,
    kg_embedding_model_name     = 'Region',
    region_type                 = ['sphere','ellipse'],
    embedding_dim               = [10, 20, 50, 70, 100],
    normalization_of_entities   = [1,2],
    init_radius                 = [0.01, 0.1, 1, 10, 50, 100],
    reg_lambda                  = [1e-1, 1e-3, 1e-5, 1e-7, 1e-9],
    loss_type                   = ['NLL'],  # ['MRL','NLL'],
    margin_loss                 = [0.0],  # [0.0, 0.02, 0.05, 0.1, 0.2, 0.3],   # für WS, max 0.5
    single_pass                 = [True] # [True, False],
    neg_factor                  = [1,2,5,10],
    learning_rate               = [1e-1, 1e-3, 1e-5],
    num_epochs                  = [50] if not TEST else [1],  
    batch_size                  = [1024],
    test_batch_size             = [4096],
    filter_negative_triples     = True,
    maximum_number_of_hpo_iters = 10, # Define the number HPO iterations
    preferred_device            = 'gpu'
)

Train and evaluate Region:

In [10]:
results = pykeen.run(
    config=config,
    output_directory=output_directory,
)

HPO Iteration:   0%|                                                  | 0/100 [00:00<?, ?it/s]

{'training_set_path': 'D:/uni/master/data/YAGO39K/Train/triple2id.tsv', 'test_set_path': 'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_positive.tsv', 'neg_test_set_path': 'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_negative.tsv', 'execution_mode': 'HPO_mode', 'random_seed': 2, 'kg_embedding_model_name': 'Region', 'region_type': 'sphere', 'embedding_dim': 70, 'normalization_of_entities': 1, 'init_radius': 0.1, 'reg_lambda': 0.1, 'loss_type': 'NLL', 'margin_loss': 0.1, 'single_pass': True, 'neg_factor': 1, 'learning_rate': 1, 'num_epochs': 50, 'batch_size': 1024, 'test_batch_size': 4096, 'filter_negative_triples': True, 'maximum_number_of_hpo_iters': 100, 'preferred_device': 'gpu', 'pykeen-version': '0.0.26-dev', 'num_entities': 46100, 'num_relations': 39}



Training epoch:  10%|████                                    | 5/50 [07:02<1:03:15, 84.35s/it]INFO:pykeen.utilities.evaluation_utils.metrics_computations:Evaluation took 0.31s seconds


MetricResults(mean_rank=None, hits_at_k=None, precision=0.4492687385740402, recall=0.19222997197053648, accuracy=0.4778680236161399, fscore=0.2692535950696188)


HPO Iteration:   1%|▍                                     | 1/100 [08:31<14:03:16, 511.08s/it]

...

MetricResults(mean_rank=None, hits_at_k=None, precision=0.49303621169916434, recall=0.011537709406166483, accuracy=0.49942916788987834, fscore=0.022547770700636943)


HPO Iteration:  22%|███████                         | 22/100 [17:16:19<134:36:09, 6212.43s/it]

{'training_set_path': 'D:/uni/master/data/YAGO39K/Train/triple2id.tsv', 'test_set_path': 'D:/uni/master/data/YAGO39K/Valid/triple2id_positive.tsv', 'neg_test_set_path': 'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_negative.tsv', 'execution_mode': 'HPO_mode', 'random_seed': 2, 'kg_embedding_model_name': 'Region', 'region_type': 'ellipse', 'embedding_dim': 20, 'normalization_of_entities': 1, 'init_radius': 0.1, 'reg_lambda': 10, 'loss_type': 'MRL', 'margin_loss': 0.2, 'single_pass': True, 'neg_factor': 5, 'learning_rate': 0.2, 'num_epochs': 50, 'batch_size': 1024, 'test_batch_size': 4096, 'filter_negative_triples': True, 'maximum_number_of_hpo_iters': 100, 'preferred_device': 'gpu', 'pykeen-version': '0.0.26-dev', 'num_entities': 46100, 'num_relations': 39}




















Training epoch:   0%|                                                  | 0/50 [00:00<?, ?it/s]

















Training epoch:   2%|▊                                      | 1/50 [01:42<1:23:52, 102.71s/it]

















Training epoch:   4%|█▌                                     | 2/50 [03:26<1:22:27, 103.08s/it]

KeyboardInterrupt: 

Check result entries:

In [ ]:
results.results.keys()

In [ ]:
results = {
    'losses': json.load(open(os.path.join(output_directory, 'losses.json'))),
    'validation_losses': json.load(open(os.path.join(output_directory, 'val_losses.json'))),
    'entity_to_embedding': json.load(open(os.path.join(output_directory, 'entities_to_embeddings.json'))),
    'relation_to_embedding': json.load(open(os.path.join(output_directory, 'relations_to_embeddings.json'))),
    'final_configuration': json.load(open(os.path.join(output_directory, 'configuration.json'))),
    'eval_summary': json.load(open(os.path.join(output_directory, 'evaluation_summary.json'))),
    'search_summary': json.load(open(os.path.join(output_directory, 'search_summary.json'))),
    'entity_to_id': json.load(open(os.path.join(output_directory, 'entity_to_id.json'))),
    'relation_to_id': json.load(open(os.path.join(output_directory, 'relation_to_id.json')))
}

In [ ]:
results['search_summary']

In [ ]:
import torch, os
import pykeen.constants as pkc
from pykeen.kge_models import get_kge_model
# Initialize KG embedding model
config = results['final_configuration']
if config['random_seed'] is not None:
    torch.manual_seed(config['random_seed'])
    
model: torch.nn.Module = get_kge_model(config=config)
model.load_state_dict(torch.load(os.path.join(output_directory, 'trained_model.pkl')))

results['trained_model'] = model 

Get the model configuration that lead to best results:

In [ ]:
results['final_configuration']

Access trained model:

In [ ]:
results['trained_model']

Visualize loss values:

In [ ]:
losses = results['losses']
epochs = np.arange(len(losses))
plt.title(r'Loss Per Epoch')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(epochs, losses)
plt.show()

In [ ]:
losses = results['validation_losses']
epochs = np.arange(len(losses))
plt.title(r'Loss Per Epoch')
plt.xlabel('epoch')
plt.ylabel('loss')
plt.plot(epochs, losses)
plt.show()

In [ ]:
losses

Get evaluation results:
 * Mean rank
 * Hits@k, k $\in$ {1,3,5,10} 

In [ ]:
results['eval_summary']

1.

    {'training_set_path': 'D:/uni/master/data/YAGO39K/Train/triple2id.tsv',
     'test_set_path': 'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_positive.tsv',
     'neg_test_set_path': 'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_negative.tsv',
     'execution_mode': 'HPO_mode',
     'random_seed': 2,
     'kg_embedding_model_name': 'Region',
     'embedding_dim': 20,
     'normalization_of_entities': 2,
     'init_radius': 5,
     'reg_lambda': 0.02,
     'margin_loss': 0,
     'learning_rate': 5,
     'num_epochs': 30,
     'batch_size': 1024,
     'test_batch_size': 1024,
     'filter_negative_triples': True,
     'maximum_number_of_hpo_iters': 20,
     'preferred_device': 'gpu',
     'pykeen-version': '0.0.26-dev',
     'num_entities': 46100,
     'num_relations': 39}


     {'mean_rank': None,
     'hits@k': None,
     'precision': [0.5817929759704251],
     'recall': [0.7386089563913696],
     'accuracy': [0.6035163257983495],
     'f1_score': 0.6508889335669357}

     30 epochs overfit

2.
   
       {'training_set_path': 'D:/uni/master/data/YAGO39K/Train/instanceOf2id.tsv', 'test_set_path': 'D:/uni/master/data/YAGO39K/Valid/subClassOf2id_positive.tsv', 'neg_test_set_path': 'D:/uni/master/data/YAGO39K/Valid/triple2id_negative.tsv', 'execution_mode': 'HPO_mode', 'random_seed': 2, 'kg_embedding_model_name': 'Region', 'embedding_dim': 100, 'normalization_of_entities': 2, 'init_radius': 0.5, 'reg_lambda': 0.2, 'margin_loss': 0, 'learning_rate': 5, 'num_epochs': 30, 'batch_size': 1024, 'test_batch_size': 1024, 'filter_negative_triples': True, 'maximum_number_of_hpo_iters': 20, 'preferred_device': 'gpu', 'pykeen-version': '0.0.26-dev', 'num_entities': 46100, 'num_relations': 39}

    MetricResults(mean_rank=None, hits_at_k=None, precision=(0.5531790499390986,), recall=(0.7401082067661822,), accuracy=(0.5707994911439476,), fscore=0.6331344448781575)

    18 epochs


3.

    {'training_set_path': 'D:/uni/master/data/YAGO39K/Train/instanceOf2id.tsv', 'test_set_path': 'D:/uni/master/data/YAGO39K/Valid/instanceOf2id_positive.tsv', 'neg_test_set_path': 'D:/uni/master/data/YAGO39K/Valid/triple2id_negative.tsv', 'execution_mode': 'HPO_mode', 'random_seed': 2, 'kg_embedding_model_name': 'Region', 'embedding_dim': 50, 'normalization_of_entities': 2, 'init_radius': 10, 'reg_lambda': 0.1, 'margin_loss': 0, 'learning_rate': 2, 'num_epochs': 30, 'batch_size': 1024, 'test_batch_size': 1024, 'filter_negative_triples': True, 'maximum_number_of_hpo_iters': 20, 'preferred_device': 'gpu', 'pykeen-version': '0.0.26-dev', 'num_entities': 46100, 'num_relations': 39}

    MetricResults(mean_rank=None, hits_at_k=None, precision=(0.5531279471864193,), recall=(0.6881559220389805,), accuracy=(0.5657435495971557,), fscore=0.6132977023847561)

    30 epochs not finished

